# Messing around with location history

In [49]:
#!pip install HASS-data-detective # Install detective

In [50]:
!pip show HASS-data-detective

Name: HASS-data-detective
Version: 2.4
Summary: Tools for studying Home Assistant data.
Home-page: https://github.com/robmarkcole/HASS-data-detective
Author: Robin Cole
Author-email: robmarkcole@gmail.com
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: pandas, pytz, ruamel.yaml, SQLAlchemy
Required-by: 


In [51]:
import detective.core as detective
import detective.functions as functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

db = detective.db_from_hass_config()

YAML tag !include_dir_merge_list is not supported
YAML tag !include_dir_merge_named is not supported
Successfully connected to database sqlite:////config/home-assistant_v2.db
There are 348 entities with data


First we extract all state changes for Gaby and Charles

In [52]:
people = ("person.charles","person.gaby",)

In [53]:
people_str = str(tuple(people))
if len(people) == 1:
    people_str = people_str.replace(",", "")
            
url = "sqlite:////config/home-assistant_v2.db"

query = f"""
    SELECT last_updated, entity_id, state, shared_attrs
    FROM states
    LEFT JOIN state_attributes ON states.attributes_id = state_attributes.attributes_id
    WHERE
        entity_id IN {people_str}
    AND
        state NOT IN ('unknown', 'unavailable')
    ORDER BY last_updated DESC
    """

results = pd.read_sql_query(query, url)
print(f"The returned Pandas dataframe has {results.shape[0]} rows of data.")
results

The returned Pandas dataframe has 26786 rows of data.


,last_updated,entity_id,state,shared_attrs
0,2022-09-03 08:53:41.164757,person.charles,home,"{""editable"":false,""id"":""charles"",""latitude"":51..."
1,2022-09-03 08:53:41.164248,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2,2022-09-03 08:52:23.431763,person.charles,home,"{""editable"":false,""id"":""charles"",""latitude"":51..."
3,2022-09-03 08:52:23.430499,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
4,2022-09-02 23:56:04.008795,person.charles,home,"{""editable"":false,""id"":""charles"",""latitude"":51..."
...,...,...,...,...
26781,2022-05-06 03:38:07.624507,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
26782,2022-05-06 03:30:02.878928,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
26783,2022-05-06 03:24:58.453512,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
26784,2022-05-06 03:20:23.946649,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."


In [54]:
results.last_updated = pd.to_datetime(results.last_updated)
results.set_index('last_updated', inplace=True)
results.index.rename("time", inplace=True)
results.sort_index(inplace=True)
results

,entity_id,state,shared_attrs
time,,,
2022-05-06 03:15:07.110833,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:20:23.946649,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:24:58.453512,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:30:02.878928,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:38:07.624507,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
...,...,...,...
2022-09-02 23:56:04.008795,person.charles,home,"{""editable"":false,""id"":""charles"",""latitude"":51..."
2022-09-03 08:52:23.430499,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-09-03 08:52:23.431763,person.charles,home,"{""editable"":false,""id"":""charles"",""latitude"":51..."


In [69]:
import yaml
l = lambda s : yaml.safe_load(s)
results["parsed_attrs"] = results.loc[:, "shared_attrs"].apply(l)

In [73]:
results['latitude'] = results['parsed_attrs'].apply(lambda i : i['latitude'])
results['longitude'] = results['parsed_attrs'].apply(lambda i : i['longitude'])

In [ ]:
results.plot.scatter(x="longitude", y="latitude", alpha=0.4, label="entity_id");